In [1]:
import numpy as np
import glob

import torch
import torch.nn as nn

from Utils.PTModel.Models import LSTMModel

MODELNAME = "TestVAEwATT"

In [10]:
gameName = "LodeRunner"
rowLength = 32
numOfRows = 22

lrEmbeddingPath = f"Models/{MODELNAME}/LevelUnifiedRep/{gameName}"
lrEmbeddingPaths = sorted(glob.glob(f"{lrEmbeddingPath}/level*.npy"))

columnRefArray = np.array([np.arange(0, 32) for i in range(numOfRows+5)]).flatten()

In [6]:
import shutil
import os

if os.path.isdir(f"Models/{MODELNAME}/LRLSTMData"):
    shutil.rmtree(f"Models/{MODELNAME}/LRLSTMData")

os.mkdir(f"Models/{MODELNAME}/LRLSTMData")

padSize = rowLength * 3 # lode runner row length * 3 as paper uses previous 3 rows for lstm
N = padSize

sosArray = np.ones(shape=(1, 256)) * 9
eosArray = np.ones(shape=(1, 256)) * 5

xTrain = []
yTrain = []
xTrainTargetIn = []
columnRef = []

batchNum = 0

for i, levelEmbeddingPath in enumerate(lrEmbeddingPaths):

    levelEmbeddingArray = np.load(levelEmbeddingPath)

    for j in range(len(levelEmbeddingArray) - N):

        padLength = (N - j) if j < N else 0
        RowCutOff = 0 if j <= N else RowCutOff+1
        # print(f"RowCutOff: {RowCutOff}")
        # print(f"j: {j}, N: {N}")
        # print(j < N)
        # print(padLength)

        dataI = np.concatenate((np.zeros(shape=(padLength, 256)), levelEmbeddingArray[RowCutOff:j]), axis=0)
        # print(f"dataI shape: {dataI.shape}")

        dataT = levelEmbeddingArray[j:j+N]
        targetIn = np.concatenate((sosArray, dataT))
        targetOut = np.concatenate((dataT, eosArray))

        #levelIdx = np.concatenate((np.zeros(shape=(padLength)), columnRefArray[RowCutOff:j]), axis=0)
        levelIdx = np.array(columnRefArray[j:N+j])
        dataC = np.zeros(shape=(N, 256))
        for t in range(N): dataC[t][int(levelIdx[t])] = 1
        # print(f"dataC shape: {dataC.shape} : {dataC[-1]}")

        columnRef.append(dataC)
        xTrain.append(dataI)
        xTrainTargetIn.append(targetIn)
        yTrain.append(targetOut)

    if len(xTrain) / 32 >= 100.0:

        savePath = f"Models/{MODELNAME}/LRLSTMData/batch{batchNum}"

        np.save(f"{savePath}xTrain.npy", np.array(xTrain))
        np.save(f"{savePath}xTrainTargetIn.npy", np.array(xTrainTargetIn))
        np.save(f"{savePath}yTrain.npy", np.array(yTrain))
        np.save(f"{savePath}columnRef.npy", np.array(columnRef))

        xTrain = []
        yTrain = []
        xTrainTargetIn = []
        columnRef = []

        batchNum += 1

if len(xTrain) > 0:

    savePath = f"Models/{MODELNAME}/LRLSTMData/batch{batchNum}"

    np.save(f"{savePath}xTrain.npy", np.array(xTrain))
    np.save(f"{savePath}xTrainTargetIn.npy", np.array(xTrainTargetIn))
    np.save(f"{savePath}yTrain.npy", np.array(yTrain))
    np.save(f"{savePath}columnRef.npy", np.array(columnRef))


# xTrain = np.array(xTrain)
# xTrainTargetIn = np.array(xTrainTargetIn)
# yTrain = np.array(yTrain)
# columnRef = np.array(columnRef)

In [25]:
def TrainModel(xTrain, xTrainTargetIn, yTrain, columnRef, epochs, batchSize):

    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = LSTMModel()
    # optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, eps=1e-7)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-7)

    criterion = nn.MSELoss()

    model.to(device)
    model.train()

    losses = []

    print(f"xTrain b4 tensor shape: {xTrain.shape}")
    #xTrain = xTrain.reshape(xTrain.shape[0], xTrain.shape[2], xTrain.shape[1])
    #xTrainTargetIn = xTrainTargetIn.reshape(xTrainTargetIn.shape[0], xTrainTargetIn.shape[2], xTrainTargetIn.shape[1])
    #yTrain = yTrain.reshape(yTrain.shape[0], yTrain.shape[2], yTrain.shape[1])
    #columnRef = columnRef.reshape(columnRef.shape[0], columnRef.shape[2], columnRef.shape[1])

    for i in range(epochs):

        losses.append([])

        for j in range(0, xTrain.shape[0], batchSize):
            
            xTrainTensor = torch.tensor(xTrain[j:j+batchSize], dtype=torch.float32).to(device)
            xTrainTargetInTensor = torch.tensor(xTrainTargetIn[j:j+batchSize], dtype=torch.float32).to(device)

            yTrainTensor = torch.tensor(yTrain[j:j+batchSize], dtype=torch.float32).to(device)

            columnRefTensor = torch.tensor(columnRef[j:j+batchSize], dtype=torch.float32).to(device)

            # print(xTrainTensor.shape)
            # print(xTrainTargetInTensor.shape)
            # print(yTrainTensor.shape)
            # print(columnRefTensor.shape)

            #print(f"xTrain size: {xTrainTensor.size()}")

            yPred = model(xTrainTensor, xTrainTargetInTensor, columnRefTensor)
            
            #print(f"yPred size: {yPred.size()}")
            #print(f"yTruth size: {yTrainTensor.size()}")
            loss = criterion(yPred, yTrainTensor)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses[i].append(loss.cpu().detach().item())

        print(f"Epoch {i}: loss {sum(losses[i])/len(losses[i])}")
    
    return model

In [ ]:
model = TrainModel(xTrain, xTrainTargetIn, yTrain, columnRef, 25, 32)

In [3]:
import Utils.fastnumpyio as fnp

def TrainModelFromFiles(batchPaths, epochs, batchSize, continueTraining=None, learningRate=0.001):

    device = "cuda" if torch.cuda.is_available() else "cpu"

    if continueTraining == None:
        model = LSTMModel()
    else:
        model = continueTraining
    
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learningRate, eps=1e-7)
    #optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, eps=1e-7)

    criterion = nn.MSELoss()

    model.to(device)
    model.train()

    losses = []

    for i in range(epochs):

        losses.append([])

        for t in range(0, len(batchPaths), 4):
            print(t)
            # print(batchPaths[t:t+4])

            xTrain = fnp.load(batchPaths[t+1])
            xTrainTargetIn = fnp.load(batchPaths[t+2])
            yTrain = fnp.load(batchPaths[t+3])
            columnRef = fnp.load(batchPaths[t])

            for j in range(0, xTrain.shape[0], batchSize):
                
                xTrainTensor = torch.tensor(xTrain[j:j+batchSize], dtype=torch.float32).to(device)
                xTrainTargetInTensor = torch.tensor(xTrainTargetIn[j:j+batchSize], dtype=torch.float32).to(device)

                yTrainTensor = torch.tensor(yTrain[j:j+batchSize], dtype=torch.float32).to(device)

                columnRefTensor = torch.tensor(columnRef[j:j+batchSize], dtype=torch.float32).to(device)

                # print(xTrainTensor.shape)
                # print(xTrainTargetInTensor.shape)
                # print(yTrainTensor.shape)
                # print(columnRefTensor.shape)

                #print(f"xTrain size: {xTrainTensor.size()}")

                yPred = model(xTrainTensor, xTrainTargetInTensor, columnRefTensor)
                
                #print(f"yPred size: {yPred.size()}")
                #print(f"yTruth size: {yTrainTensor.size()}")
                loss = criterion(yPred, yTrainTensor)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                losses[i].append(loss.cpu().detach().item())

            print(f"Epoch {i} Batch {t}: loss {losses[i][-1]}")

        print(f"Epoch {i}: loss {sum(losses[i])/len(losses[i])}")
    
    return model

In [5]:
model = TrainModelFromFiles(sorted(glob.glob(f"Models/{MODELNAME}/LRLSTMData/*")), 20, 32, learningRate=0.001, continueTraining=model)

0
Epoch 0 Batch 0: loss 0.7844229936599731
4
Epoch 0 Batch 4: loss 0.4697379469871521
8
Epoch 0 Batch 8: loss 0.5302704572677612
12
Epoch 0 Batch 12: loss 0.40791139006614685
16
Epoch 0 Batch 16: loss 0.667203426361084
20
Epoch 0 Batch 20: loss 0.6138553023338318
24
Epoch 0 Batch 24: loss 0.8279170393943787
28
Epoch 0 Batch 28: loss 0.4224819242954254
32
Epoch 0 Batch 32: loss 0.4135683476924896
36
Epoch 0 Batch 36: loss 0.7125810980796814
40
Epoch 0 Batch 40: loss 0.5643162727355957
44
Epoch 0 Batch 44: loss 0.5660424828529358
48
Epoch 0 Batch 48: loss 0.5085227489471436
52
Epoch 0 Batch 52: loss 0.5907588005065918
56
Epoch 0 Batch 56: loss 0.7101013660430908
60
Epoch 0 Batch 60: loss 0.6770675182342529
64
Epoch 0 Batch 64: loss 0.6672075390815735
68
Epoch 0 Batch 68: loss 0.7162806987762451
72
Epoch 0 Batch 72: loss 0.7146070003509521
76
Epoch 0 Batch 76: loss 0.514797568321228
80
Epoch 0 Batch 80: loss 0.6037762761116028
84
Epoch 0 Batch 84: loss 0.6199766397476196
88
Epoch 0 Batch 

In [6]:
torch.save(model, f"Models/{MODELNAME}/LodeRunnerLSTMFullDataset.pt")

In [27]:
total = 0
for name, param in model.named_parameters():
    print(f"{name:<24}: {param.numel():5}")
    total += param.numel()

print(f"Total Params: {total}")

histLSTM.weight_ih_l0   : 131072
histLSTM.weight_hh_l0   : 65536
histLSTM.bias_ih_l0     :   512
histLSTM.bias_hh_l0     :   512
colLSTM.weight_ih_l0    : 131072
colLSTM.weight_hh_l0    : 65536
colLSTM.bias_ih_l0      :   512
colLSTM.bias_hh_l0      :   512
textLSTM.weight_ih_l0   : 131072
textLSTM.weight_hh_l0   : 65536
textLSTM.bias_ih_l0     :   512
textLSTM.bias_hh_l0     :   512
infTextLSTM.weight_ih_l0: 131072
infTextLSTM.weight_hh_l0: 65536
infTextLSTM.bias_ih_l0  :   512
infTextLSTM.bias_hh_l0  :   512
outputLayer.weight      : 32768
outputLayer.bias        :   256
Total Params: 823552
